In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os

# ===============================
# 1. Crear carpeta results si no existe
# ===============================
if not os.path.exists("results"):
    os.makedirs("results")

# ===============================
# 2. Cargar y preparar datos
# ===============================
df = pd.read_excel("data/consolidated/consolidado_salud.xlsx")

# Asegurarse de que PRECIO_UNITARIO sea numérico
df['PRECIO_UNITARIO'] = pd.to_numeric(df['PRECIO_UNITARIO'], errors='coerce')

# Logaritmo del precio unitario y volatilidad para visualización interna
df['PRECIO_UNITARIO_LOG'] = np.log1p(df['PRECIO_UNITARIO'])

# ===============================
# 3. Gráficos de precio
# ===============================

# 3.1 Conteo de categorías por Fuente (Barplot)
fig1 = px.histogram(df, x='CATEGORIA', color='FUENTE', barmode='group',
                    labels={'CATEGORIA':'Categoría', 'count':'Cantidad de productos', 'FUENTE':'Fuente'})
fig1.update_layout(xaxis_tickangle=-45)
fig1.write_image("results/cantidad_productos_por_categoria.pdf")
fig1.show()

# 3.2 Precio promedio por categoría y fuente (Heatmap)
precio_categoria_fuente = df.groupby(['CATEGORIA','FUENTE'])['PRECIO_UNITARIO_LOG'].mean().reset_index()
fig2 = px.density_heatmap(precio_categoria_fuente, x='FUENTE', y='CATEGORIA',
                          z='PRECIO_UNITARIO_LOG', color_continuous_scale='Blues',
                          labels={'PRECIO_UNITARIO_LOG':'Precio Unitario', 'CATEGORIA':'Categoría','FUENTE':'Fuente'})
fig2.write_image("results/precio_promedio_por_categoria.pdf")
fig2.show()

# ===============================
# 4. Gráficos de volatilidad
# ===============================

# Calcular volatilidad por producto y fuente
volatilidad_df = df.groupby(['CLAVE_MATCHING','FUENTE']).agg(
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO','std')
).reset_index()

# Logaritmo de precio y volatilidad para visualización interna
volatilidad_df['LOG_PRECIO'] = np.log1p(volatilidad_df['PRECIO_PROMEDIO'])
volatilidad_df['LOG_VOLATILIDAD'] = np.log1p(volatilidad_df['VOLATILIDAD'])

# 4.1 Scatterplot Precio vs Volatilidad
fig3 = px.scatter(volatilidad_df, x='LOG_PRECIO', y='LOG_VOLATILIDAD', color='FUENTE',
                  labels={'LOG_PRECIO':'Precio Unitario', 'LOG_VOLATILIDAD':'Volatilidad', 'FUENTE':'Fuente'})
fig3.write_image("results/precio_vs_volatilidad.pdf")
fig3.show()

# 4.2 Heatmap de volatilidad promedio por categoría y fuente
vol_categoria_fuente = df.groupby(['CATEGORIA','FUENTE'])['PRECIO_UNITARIO'].std().reset_index()
vol_categoria_fuente['VOL_LOG'] = np.log1p(vol_categoria_fuente['PRECIO_UNITARIO'])

fig4 = px.density_heatmap(vol_categoria_fuente, x='FUENTE', y='CATEGORIA', z='VOL_LOG',
                          color_continuous_scale='YlOrRd',
                          labels={'VOL_LOG':'Volatilidad','CATEGORIA':'Categoría','FUENTE':'Fuente'})
fig4.write_image("results/volatilidad_promedio_por_categoria.pdf")
fig4.show()

# ===============================
# 5. Tablas de volatilidad y precio
# ===============================

# Función para graficar tabla y guardar en PDF
def guardar_tabla_pdf(df_tabla, nombre_archivo, titulo=""):
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df_tabla.columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[df_tabla[col] for col in df_tabla.columns],
                   fill_color='lavender',
                   align='left'))
    ])
    fig.update_layout(title_text=titulo)
    fig.write_image(f"results/{nombre_archivo}.pdf")
    fig.show()

# Volatilidad por FUENTE y TIPO_ENVASE
tabla_envase_fuente = df.groupby(['TIPO_ENVASE','FUENTE']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO','std')
).reset_index()
display(tabla_envase_fuente)

# Volatilidad por FUENTE y CATEGORIA
tabla_categoria_fuente = df.groupby(['CATEGORIA','FUENTE']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO','std')
).reset_index()
display(tabla_categoria_fuente)

# Precio promedio por FUENTE y TIPO_ENVASE
tabla_precio_envase = df.groupby(['TIPO_ENVASE','FUENTE']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    PRECIO_PROMEDIO_LOG=('PRECIO_UNITARIO_LOG','mean')
).reset_index()
display(tabla_precio_envase)

# Precio promedio por FUENTE y CATEGORIA
tabla_precio_categoria = df.groupby(['CATEGORIA','FUENTE']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    PRECIO_PROMEDIO_LOG=('PRECIO_UNITARIO_LOG','mean')
).reset_index()
display(tabla_precio_categoria)


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os

# ===============================
# 1. Crear carpeta results si no existe
# ===============================
if not os.path.exists("results"):
    os.makedirs("results")

# ===============================
# 2. Cargar y preparar datos
# ===============================
df = pd.read_excel("data/consolidated/consolidado_salud.xlsx")
df['PRECIO_UNITARIO'] = pd.to_numeric(df['PRECIO_UNITARIO'], errors='coerce')
df['PRECIO_UNITARIO_LOG'] = np.log1p(df['PRECIO_UNITARIO'])

# Calcular volatilidad por producto (general, sin fuente)
vol_df = df.groupby('CLAVE_MATCHING').agg(
    PRECIO_PROMEDIO=('PRECIO_UNITARIO_LOG','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO_LOG','std')
).reset_index()
vol_df['VOLATILIDAD'] = vol_df['VOLATILIDAD'].fillna(0)

# ===============================
# 3. Escalar variables para clustering
# ===============================
scaler = StandardScaler()
X = scaler.fit_transform(vol_df[['PRECIO_PROMEDIO','VOLATILIDAD']])

# ===============================
# 4. Clustering K-Means (solo precio y volatilidad)
# ===============================
k = 3  # Número de clusters
kmeans = KMeans(n_clusters=k, random_state=42)
vol_df['CLUSTER'] = kmeans.fit_predict(X)

# ===============================
# 5. Gráficos de clustering (general)
# ===============================

# Scatterplot Precio vs Volatilidad con clusters
fig1 = px.scatter(
    vol_df,
    x='PRECIO_PROMEDIO', y='VOLATILIDAD',
    color='CLUSTER',
    labels={'PRECIO_PROMEDIO':'Precio Unitario',
            'VOLATILIDAD':'Volatilidad',
            'CLUSTER':'Cluster'}
)
fig1.write_image("results/clusters_precio_volatilidad.pdf")
fig1.show()

# Distribución de clusters (histograma general)
cluster_count = vol_df.groupby('CLUSTER').size().reset_index(name='CANTIDAD')
fig2 = px.bar(cluster_count, x='CLUSTER', y='CANTIDAD',
              labels={'CLUSTER':'Cluster','CANTIDAD':'Cantidad de productos'})
fig2.write_image("results/distribucion_clusters.pdf")
fig2.show()

# ===============================
# 6. Tablas de clusters (solo mostrar)
# ===============================

# Cluster vs Categoría (sin fuente)
tabla_cluster_categoria = df.merge(vol_df[['CLAVE_MATCHING','CLUSTER']], on='CLAVE_MATCHING')
tabla_cluster_categoria = tabla_cluster_categoria.groupby(['CLUSTER','CATEGORIA']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO','std')
).reset_index()
print("Tabla de clusters por Categoría (general):")
display(tabla_cluster_categoria)

# Cluster vs Tipo de envase (sin fuente)
tabla_cluster_envase = df.merge(vol_df[['CLAVE_MATCHING','CLUSTER']], on='CLAVE_MATCHING')
tabla_cluster_envase = tabla_cluster_envase.groupby(['CLUSTER','TIPO_ENVASE']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO','std')
).reset_index()
print("\nTabla de clusters por Tipo de Envase (general):")
display(tabla_cluster_envase)
